# MCP Clients

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to LLMs.

In AG2, you can build a toolkit from the MCP tools available in an MCP session and these tools will be made available to the AG2 clients.

This guide explains how to build MCP clients that connect to MCP servers.

## Installation

**Note: Python 3.10 and higher is required.**

To integrate MCP tools into the AG2 framework, install the required dependencies (`openai` installs the OpenAI client for the examples in this notebook):

```bash
pip install -U ag2[openai,mcp]
```

> **Note:** If you have been using `autogen` or `pyautogen`, all you need to do is upgrade it using:  
> ```bash
> pip install -U autogen[openai,mcp]
> ```
> or  
> ```bash
> pip install -U pyautogen[openai,mcp]
> ```
> as `pyautogen`, `autogen`, and `ag2` are aliases for the same PyPI package.  

## Imports

In [ ]:
from pathlib import Path

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from autogen.agentchat import AssistantAgent, ConversableAgent
from autogen.mcp import create_toolkit

In [ ]:
# Only needed for Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

## Setting up an MCP Server

We will create a simple MCP server that exposes two tools: `add` and `multiply`.

For more details on creating MCP servers, visit the [MCP Python SDK documentation](https://github.com/modelcontextprotocol/python-sdk).

Let's create a Python script (`math_server.py`) with the following content:

In [ ]:
# Our server is a Python script that has two tools in it:
# - add: adds two numbers
# - multiply: multiplies two numbers
# The docstrings will be utilised to describe the tools for our agents
math_server_file_content = """# math_server.py
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Math")


@mcp.tool()
def add(a: int, b: int) -> int:
    \"\"\"Add two numbers\"\"\"
    return a + b


@mcp.tool()
def multiply(a: int, b: int) -> int:
    \"\"\"Multiply two numbers\"\"\"
    return a * b


if __name__ == "__main__":
    mcp.run(transport="stdio")
"""

# Write content to a file
math_server_path = Path("math_server.py")
math_server_path.write_text(math_server_file_content)

## Connecting AG2 to the MCP Server

To connect AG2 to the MCP server, follow these steps:
- **Set up connection parameters** – Define how AG2 will communicate with the MCP server.
- **Start a client session** – This session handles communication between AG2 and the server.
- **Wrap MCP tools into a toolkit** – This makes them accessible to AG2.
- **Register the toolkit with an AG2 agent** – This enables AG2 to use MCP tools.
- **Start conversation** - Send a message with desired goal (e.g. adding two numbers).

In [ ]:
# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command="python",  # Executable
    args=[str(math_server_path)],  # Optional command line arguments
)

async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    # Initialize the connection
    await session.initialize()

    # Create an AG2 toolkit with available MCP tools
    toolkit = await create_toolkit(session=session)
    agent = AssistantAgent(name="assistant", llm_config={"model": "gpt-4o-mini", "api_type": "openai"})

    # Register MCP tools with the agent
    toolkit.register_for_llm(agent)

    # Ask our agent to add two numbers (where it will choose the correct tool)
    result = await agent.a_run(
        message="Add 123223 and 456789",
        tools=toolkit.tools,
        max_turns=2,
        user_input=False,
    )
    await result.process()

In the following example we'll have our agent utilize both of the tools in the tool kit, add and multiply.

The agent will call add first, get the result, then call the multiply function with the result and 92 to get the final answer.

In [ ]:
# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command="python",  # Executable
    args=[str(math_server_path)],  # Optional command line arguments
)

async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    # Initialize the connection
    await session.initialize()

    # Create an AG2 toolkit with available MCP tools
    toolkit = await create_toolkit(session=session)

    # We'll turn off parallel tool calls so that the agent doesn't try to call both tools at the same time
    # Reason: it needs the answer from the add tool before calling the multiply tool
    agent = ConversableAgent(
        name="assistant", llm_config={"model": "gpt-4o-mini", "api_type": "openai", "parallel_tool_calls": False}
    )

    # Register MCP tools with the agent
    toolkit.register_for_llm(agent)

    # Ask our agent to add two numbers
    result = await agent.a_run(
        message="What is (123223 + 456789) * 92?",
        tools=toolkit.tools,
        max_turns=3,
        user_input=False,
    )
    await result.process()